In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import *
import time
import datetime
import scipy.stats as st
from fitter import Fitter

import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv('demand_ups.csv')
data.head()

,Unnamed: 0,Lane,date,real_d,predict_d,weekday,month,year,quarter,period
0,0,SCN-SZX HUB (US),2019-01-01,0.0,8247.0,2.0,1.0,2019.0,1.0,1.0
1,1,SCN-SZX HUB (US),2019-01-02,0.0,131205.0,3.0,1.0,2019.0,1.0,1.0
2,2,SCN-SZX HUB (US),2019-01-03,589810.0,334451.0,4.0,1.0,2019.0,1.0,1.0
3,3,SCN-SZX HUB (US),2019-01-04,501838.0,315303.0,5.0,1.0,2019.0,1.0,1.0
4,4,SCN-SZX HUB (US),2019-01-05,616311.0,402192.0,6.0,1.0,2019.0,1.0,1.0


In [3]:
#We just select SZX-US and HK-US two lanes
#input: 1,3,SZX/US,(default:2019),predict/real

# eg. df=data_select(1,3,2019,'SZX','predict')

lane_dict={'SZX':'SCN-SZX HUB (US)','HK':'HK HUB (US)'}
type_dict={'predict':'predict_d','real':'real_d'}

def data_select(weekday,period,year,lane,datatype='predict'):   
    df=data[(data['Lane']==lane_dict[lane]) & (data['weekday']==weekday) 
            &(data['period']==period) &(data['year']==year)]
    column=['Lane', 'date',type_dict[datatype] ,'weekday', 'month', 'year',
       'quarter', 'period']
    df=df[column]
    df=df[(df[type_dict[datatype]]>=1000)]
    df.reset_index(drop=True,inplace=True)
    df=df.rename(columns={type_dict[datatype]:'demand'})
    return df

In [4]:
df=data_select(1,3,2019,'SZX','predict')
df.head(5)

,Lane,date,demand,weekday,month,year,quarter,period
0,SCN-SZX HUB (US),2019-10-14,374052.0,1.0,10.0,2019.0,4.0,3.0
1,SCN-SZX HUB (US),2019-10-21,382281.0,1.0,10.0,2019.0,4.0,3.0
2,SCN-SZX HUB (US),2019-10-28,378191.0,1.0,10.0,2019.0,4.0,3.0
3,SCN-SZX HUB (US),2019-11-04,377295.0,1.0,11.0,2019.0,4.0,3.0
4,SCN-SZX HUB (US),2019-11-11,554677.0,1.0,11.0,2019.0,4.0,3.0


In [5]:
df=data_select(1,3,2019,'SZX','real')
df.head(5)
x_predict=0

In [6]:
def flexibility_model(demand,c,cost_over,cost_under):
    ######### Model Set-up ############
    m = Model()
    M=1e20

    #flexibility
    x = m.addVar(vtype=GRB.CONTINUOUS,name = "flexibility")#x
    y1= m.addVars(2,vtype=GRB.CONTINUOUS,name = "y1")
    y2= m.addVars(2,vtype=GRB.CONTINUOUS,name = "y2")
    z1 = m.addVars(2,2,vtype=GRB.BINARY,name='z1')
    z2 = m.addVars(2,2,vtype=GRB.BINARY,name='z2')

    # set objective
    m.setObjective( quicksum((cost_over[t]*y1[t]+cost_under[t]*y2[t]) for t in range(2)), GRB.MINIMIZE)#

    # add constraint
    
    m.addConstrs( y1[t]>=c[t]-demand[t]+x*((-1)**t)  for t in range(2))
    m.addConstrs( y1[t]>=0 for t in range(2))
    m.addConstrs( y1[t]<=c[t]-demand[t]+x*((-1)**t)+M*(1-z1[0,t]) for t in range(2))
    m.addConstrs( y1[t]<=M*(1-z2[0,t])  for t in range(2))
    m.addConstrs( z1[0,t]+z2[0,t]>=1 for t in range(2))
    
    m.addConstrs( y2[t]>=-c[t]+demand[t]-x*((-1)**t)  for t in range(2))
    m.addConstrs( y2[t]>=0 for t in range(2))
    m.addConstrs( y2[t]<=-c[t]+demand[t]-x*((-1)**t)+M*(1-z1[1,t]) for t in range(2))
    m.addConstrs( y2[t]<=M*(1-z2[1,t])  for t in range(2))
    m.addConstrs( z1[1,t]+z2[1,t]>=1 for t in range(2))

    
    #m.addConstr( x<=100000 )
    m.addConstr( x>=0 )
    m.addConstr( x<=max(demand[0]-c[0],0) )

    # Supressing the optimization output
    m.setParam( 'OutputFlag', False )

    # Solving the model
    m.optimize()
    
    return m,x

In [7]:
def flexibility_mode_actual(demand,c,cost_over,cost_under,x_predict):
    ######### Model Set-up ############
    m = Model()
    M=1e20

    #flexibility
    x = m.addVar(vtype=GRB.CONTINUOUS,name = "flexibility")#x
    y1= m.addVars(2,vtype=GRB.CONTINUOUS,name = "y1")
    y2= m.addVars(2,vtype=GRB.CONTINUOUS,name = "y2")
    z1 = m.addVars(2,2,vtype=GRB.BINARY,name='z1')
    z2 = m.addVars(2,2,vtype=GRB.BINARY,name='z2')

    # set objective
    m.setObjective( quicksum((cost_over[t]*y1[t]+cost_under[t]*y2[t]) for t in range(2)), GRB.MINIMIZE)#

    # add constraint
    
    m.addConstrs( y1[t]>=c[t]-demand[t]+x*((-1)**t)  for t in range(2))
    m.addConstrs( y1[t]>=0 for t in range(2))
    m.addConstrs( y1[t]<=c[t]-demand[t]+x*((-1)**t)+M*(1-z1[0,t]) for t in range(2))
    m.addConstrs( y1[t]<=M*(1-z2[0,t])  for t in range(2))
    m.addConstrs( z1[0,t]+z2[0,t]>=1 for t in range(2))
    
    m.addConstrs( y2[t]>=-c[t]+demand[t]-x*((-1)**t)  for t in range(2))
    m.addConstrs( y2[t]>=0 for t in range(2))
    m.addConstrs( y2[t]<=-c[t]+demand[t]-x*((-1)**t)+M*(1-z1[1,t]) for t in range(2))
    m.addConstrs( y2[t]<=M*(1-z2[1,t])  for t in range(2))
    m.addConstrs( z1[1,t]+z2[1,t]>=1 for t in range(2))

    
    #m.addConstr( x<=100000 )
    m.addConstr( x>=0 )
    m.addConstr( x<=max(demand[0]-c[0],0) )
    m.addConstr( x<=x_predict)

    # Supressing the optimization output
    m.setParam( 'OutputFlag', False )

    # Solving the model
    m.optimize()
    
    return m,x

In [27]:
def outcome(weekday,period,year):
    weekday_x=weekday
    period_x=period
    year_x=year

    SZXUS=data_select(weekday_x,period_x,year_x,'SZX','predict')
    HKUS=data_select(weekday_x,period_x,year_x,'HK','predict')
    
    df2=HKUS.copy()
    df1=SZXUS.copy()

    # continuous distribution (fit)
    mu1,sigma1=st.norm.fit(df1['demand'])
    print('mu and sigma for SZ:', mu1,sigma1)

    mu2,sigma2=st.norm.fit(df2['demand'])
    print('mu and sigma for HK:',mu2,sigma2)
    print('='*50)

    rv1=st.norm(loc=mu1,scale=sigma1)
    xmin1=min(df1['demand'])
    xmax1=max(df1['demand'])
    x1=np.linspace(xmin1,xmax1,10000)

    rv2=st.norm(loc=mu2,scale=sigma2)
    xmin2=min(df2['demand'])
    xmax2=max(df2['demand'])
    x2=np.linspace(xmin2,xmax2,10000)

    #cost
    cost_over=np.array([7.5,5.7])
    cost_under=np.array([9,7.2])

    value1=(cost_over[0]-0)/(cost_over[0]+cost_under[0])
    c_opt0=rv1.ppf(value1)
    print('non flexible capacity for SZ:',c_opt0)

    value2=(cost_over[1]-0)/(cost_over[1]+cost_under[1])
    c_opt1=rv2.ppf(value2)
    print('non flexible capacity for HK:',c_opt1)
    print('='*50)

    #non-flexible system
    ## predicted cost
    d1=SZXUS['demand']
    d2=HKUS['demand']
   
    cost0=0
    cost1=0

    for j in range(len(d1)):
        cost0+=cost_over[0]*max(c_opt0-d1[j],0)+cost_under[0]*max(d1[j]-c_opt0,0)
    for j in range(len(d2)):
        cost1+=cost_over[1]*max(c_opt1-d2[j],0)+cost_under[1]*max(d2[j]-c_opt1,0)
    cost=cost0+cost1

    print('predict cost for SZ:',cost0)
    print('predict cost for HK:',cost1)
    print('predict non-flexible total cost:',cost)
    print('='*50)
    
    ##actual cost
    d11=data_select(weekday_x,period_x,year_x,'SZX','real')
    d12=data_select(weekday_x,period_x,year_x,'HK','real')
    d11_1=d11['demand']
    d12_1=d12['demand']
    cost2=0
    cost3=0

    for j in range(len(d11_1)):
        cost2+=cost_over[0]*max(c_opt0-d11_1[j],0)+cost_under[0]*max(d11_1[j]-c_opt0,0)
    for j in range(len(d12_1)):
        cost3+=cost_over[1]*max(c_opt1-d12_1[j],0)+cost_under[1]*max(d12_1[j]-c_opt1,0)
    cost_1=cost2+cost3

    print('actual cost for SZ:',cost2)
    print('actual cost for HK:',cost3)
    print('actual non-flexible total cost:',cost_1)
    print(cost2)
    print(cost3)
    print('='*50)

    #flexible system
    df21=SZXUS[['date','demand']]
    df21.rename(columns={'demand':'SZX_demand'},inplace=True)
    df22=HKUS[['date','demand']]
    df22.rename(columns={'demand':'HK_demand'},inplace=True)
    df_flex=pd.merge(df21,df22,how='outer',on='date')
    df_flex=df_flex.fillna(0)
    
    c=[c_opt0, c_opt1]
    
    #cost
    cost_over=np.array([7.5,5.7])
    cost_under=np.array([9,7.2])

    ## get predicted flexibility
    demand=np.array(df_flex[['SZX_demand','HK_demand']])
    
    for i in range(len(demand)):
        m,x=flexibility_model(demand[i],c,cost_over,cost_under)
        df_flex.loc[i,'x_predict']=x.x
        if df_flex.loc[i,'SZX_demand']==0:
            df_flex.loc[i,'ojbvalue']=cost_over[1]*max(c[1]-df_flex.loc[i,'HK_demand'],0)+cost_under[1]*max(df_flex.loc[i,'HK_demand']-c[1],0)
            #df_flex.loc[i,'real_cost']=cost_over[1]*max(c[1]-df_flex.loc[i,'HK_demand_real'],0)+cost_under[1]*max(df_flex.loc[i,'HK_demand_real']-c[1],0)
        else :
            df_flex.loc[i,'ojbvalue']=m.objVal
            #df_flex.loc[i,'real_cost']=cost_over[1]*max(c[1]-df_flex.loc[i,'HK_demand_real']-x.x,0)+cost_under[1]*max(df_flex.loc[i,'HK_demand_real']+x.x-c[1],0)\
           # +cost_over[0]*max(c[0]-df_flex.loc[i,'SZX_demand_real']+x.x,0)+cost_under[0]*max(df_flex.loc[i,'SZX_demand_real']-x.x-c[0],0)
    
    ## calculate real flexibility
    df_a=d11[['date','demand']]
    df_a.rename(columns={'demand':'SZX_demand_real'},inplace=True)
    df_b=d12[['date','demand']]
    df_b.rename(columns={'demand':'HK_demand_real'},inplace=True)
    df_flex=pd.merge(df_flex,df_a,how='outer',on='date')
    df_flex=pd.merge(df_flex,df_b,how='outer',on='date')
    df_flex=df_flex.fillna(0)

    #demand1=np.array(df_flex[['SZX_demand_real','HK_demand_real']])
    
    for i in range(len(df_flex)):
        x_predict=df_flex.loc[i,'x_predict']
        demand1=df_flex.loc[i,'SZX_demand_real']
        demand2=df_flex.loc[i,'HK_demand_real']
        demand=[demand1,demand2]
        m,x=flexibility_mode_actual(demand,c,cost_over,cost_under,x_predict)
        df_flex.loc[i,'x_real']=x.x
        if df_flex.loc[i,'SZX_demand_real']==0:
            if df_flex.loc[i,'HK_demand_real']==0:
                df_flex.loc[i,'real_cost']=0
            else:
                df_flex.loc[i,'real_cost']=cost_over[1]*max(c[1]-df_flex.loc[i,'HK_demand_real'],0)+cost_under[1]*max(df_flex.loc[i,'HK_demand_real']-c[1],0)
        else :
            df_flex.loc[i,'real_cost']=m.objVal    
       
    
    obj_cost=df_flex['ojbvalue'].sum()  
    print('predicted flexible obj cost:',obj_cost)
    print('='*50)
    
    real_cost=df_flex['real_cost'].sum()
    print('actual flexible obj cost:',real_cost)
    print('='*50)
    
    print(df_flex)

In [29]:
#for y in [2020]:
for y in (2019,2020):
    for j in (1,2,3):
        for i in (1,6):
            print('year',y,',period',j,',day',i)
            print(outcome(i,j,y))
            print("="*30,"*"*15,"="*30)
            print("   "*20)
            print("="*30,"*"*15,"="*30)

year 2019 ,period 1 ,day 1
mu and sigma for SZ: 245055.45454545456 95736.91231263427
mu and sigma for HK: 495417.3333333333 177741.5123699457
non flexible capacity for SZ: 234123.70703561165
non flexible capacity for HK: 469421.93794890126
predict cost for SZ: 6949331.501737222
predict cost for HK: 11476130.35983741
predict non-flexible total cost: 18425461.86157463
actual cost for SZ: 8373419.501737222
actual cost for HK: 14241823.458459888
actual non-flexible total cost: 22615242.96019711
8373419.501737222
14241823.458459888
predicted flexible obj cost: 17568967.24288744
actual flexible obj cost: 21150154.49006438
          date  SZX_demand  HK_demand      x_predict      ojbvalue  \
0   2019-01-07    335161.0   476872.0  101037.292964  7.811090e+05   
1   2019-01-14    344049.0   510309.0  109925.292964  1.085849e+06   
2   2019-01-21    350900.0   546986.0  116776.292964  1.399251e+06   
3   2019-01-28     10390.0   719708.0       0.000000  3.480062e+06   
4   2019-02-04    145532.0